<a href="https://colab.research.google.com/github/FinnRobertson15/PatternAnalysis-2023/blob/topic-recognition/Colab%20version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AD_NC

/content/drive/MyDrive/AD_NC


In [3]:
%ls

NC.zip  test/  train/


In [ ]:
!unzip NC.zip

unzip:  cannot find or open NC.zip, NC.zip.zip or NC.zip.ZIP.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import time
import numpy as np
# import pandas as pd
import math
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset, ConcatDataset
import random

In [118]:
batch_size = 32

In [194]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

def get_indexes_by_value(lst):
    index_dict = {}
    for i, value in enumerate(lst):
        value = value.split('_')[1].split('.jpeg')[0]
        if value in index_dict:
            index_dict[value].append(i)
        else:
            index_dict[value] = [i]

    return list(index_dict.values())

class SiameseDataset(Dataset):
    def __init__(self, AD, NC):
        # Combine the datasets and labels
        self.X = AD + NC

        self.Y = torch.cat((torch.ones(len(AD)), torch.zeros(len(NC))), dim=0)

        paths = ([str(path) for path in (AD.image_paths + NC.image_paths)])
        indexes = get_indexes_by_value(paths)
        i_indices = [random.sample(indices, len(indices)) for indices in indexes]
        j_indices = [random.sample(indices, len(indices)) for indices in indexes]
        self.i_indices = [item for sublist in i_indices for item in sublist]
        self.j_indices = [item for sublist in j_indices for item in sublist]

    def __len__(self):
        return len(self.X) // 2

    def __getitem__(self, idx):
        i = self.i_indices[idx]
        j = self.j_indices[idx]
        img1 = self.X[i]
        img2 = self.X[j]
        l1 = self.Y[i]
        l2 = self.Y[j]

        return img1, img2, l1, l2

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


size = 256

def intensity_normalization(img):
    mean = torch.mean(img)
    std = torch.std(img)
    return (img - mean) / std

def windowing(img, window_center, window_width):
    img = torch.clamp(img, window_center - window_width // 2, window_center + window_width // 2)
    img = (img - (window_center - 0.5)) / (window_width - 1)
    return img

# Example usage in your transform
transform_X = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
    transforms.Lambda(intensity_normalization),
    transforms.Lambda(lambda x: windowing(x, window_center=size // 2, window_width=size))
])


# Replace 'your_nii_folder' with the path to your folder containing .nii files
# nii_dataset = NiiDataset(root_dir=r'keras_png_slices_data\keras_png_slices_data\keras_png_slices_train')
loaders = {}

for stage in ['test']:
    loaders[stage] = {}
    AD = CustomDataset(root_dir=os.path.join(stage, 'AD'), transform=transform_X)
    NC = CustomDataset(root_dir=os.path.join(stage, 'NC'), transform=transform_X)
    dataset = SiameseDataset(AD, NC)
    loaders[stage] = DataLoader(dataset, batch_size=batch_size)

In [209]:
import torch
import torch.nn as nn
import torchvision.models as models

class SiameseNetwork(nn.Module):
    def __init__(self, pretrained=True):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet18(pretrained=pretrained)
        # Modify the first convolution layer to accept single-channel input
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=3, bias=False)
        self.fc1 = nn.Linear(1000, 500)  # Customize the fully connected layers
        self.fc2 = nn.Linear(500, 2)  # Customize the fully connected layers

    def forward(self, x1, x2):
        output1 = self.resnet(x1)
        output2 = self.resnet(x2)
        output = torch.abs(output1 - output2)
        output = self.fc1(output)
        output = self.fc2(output)
        return output

model = SiameseNetwork()

class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [210]:
# Initialize the network, loss function, and optimizer
siamese_net = SiameseNetwork(pretrained=True).to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(siamese_net.parameters(), lr=0.001)
# Training loop
epochs = 5
total_step = len(loaders['test'])
for epoch in range(epochs):
  for i, (img1, img2, lab1, lab2) in enumerate(loaders['test']):
    img1, img2, lab1, lab2 = img1.to(device), img2.to(device), lab1.to(device), lab2.to(device)
    output = siamese_net(img1, img2)
    o1, o2 = output[:, 0], output[:, 1]
    label = (lab1 == lab2).int()

    loss = criterion(o1, o2, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 21 == 0:
      print((lab1[0], lab2[0]))
      print(label[0])
      print(output[0])
      print(f"Epoch [{epoch + 1} / {epochs}], Step [{i + 1} / {total_step} Loss {loss.item()}]")

RuntimeError: ignored

In [200]:
correct = 0
total = 0
features = []
labels = []
with torch.no_grad():
    for i, (img1, img2, lab1, lab2) in enumerate(loaders['test']):
        img1, img2, lab1, lab2 = img1.to(device), img2.to(device), lab1.to(device), lab2.to(device)
        output = siamese_net(img1, img2)
        out1, out2 = output[:, 0], output[:, 1]
        predicted = (out1 - out2).pow(2).sum().sqrt().lt(0.5)
        total += lab1.size(0)
        correct += (predicted == lab1).sum().item()
        features.extend(out1.tolist())
        labels.extend(lab1.tolist())
        features.extend(out2.tolist())
        labels.extend(lab2.tolist())


    print(f'Accuracy of the network on the test images: {100 * correct / total}%')


Accuracy of the network on the test images: 49.955555555555556%


In [201]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(np.array(features).reshape(-1, 1), labels, test_size=0.2, random_state=42)

# Train a logistic regression model
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Make predictions
predictions = classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.49166666666666664


In [148]:
features[0].tolist()

0.5943959355354309

In [139]:
correct, total

(2226, 4500)